In [1]:
import os
import re
import boto3
import json

import pandas as pd
import numpy as np

from sagemaker.session import Session

# Extract Asset and Liability Line Items
**We divide asset and liability line items according to a regex parse**

In [2]:
def bsSplit(array: np.ndarray) -> tuple:
    """
    Function splits an array by bisection, into asset and liability & equity terms. Assumes that line items
    are recorded according to standard accounting practices in orientation. 
    ------------------------------------------------------------------------------------
    :param: (type numpy.ndarray)
        An array of balance sheet line items for a given broker-dealer
        
    :return: (type tuple)
        Return a tuple of arrays; index 1 is the asset items, index 2 is the liability & equity items and
        index 3 is the index where the split occured
        
    NOTE: We make the assumption that liability line items always fall below asset line items and each 
          balance sheet has both asset and liability line items. If either are missing we avoid balance sheet
    """
    stop_idx1 = 0
    stop_idx2 = array.size        # default to length of line items array
    
    asset_idx = 0        # asset and liability identifiers
    liable_idx = 0
    
    # iterate through the line items as provided by the array
    for i, item in enumerate(array):
        # search string for presence of word 'assets' and liabilites
        val1 = re.search('assets', item, flags=re.I)
        val2 = re.search('liability|liabilities', item, flags=re.I)
        
        # if we find the term "asset" we count this index
        if val1 is not None:
            asset_idx = i + 1    
        
        # if we find the term "liability" we count this index
        if val2 is not None:
            liable_idx = i + 1
        
        # in the event we find an asset and liability term, we check to see if the asset index appears before
        # the liability index, this is to prevent the JP Morgan 2012/2013 Textract Error
        if (asset_idx != 0) and (liable_idx != 0):
            
            if asset_idx < liable_idx:
                stop_idx1 = asset_idx
                stop_idx2 = liable_idx
                
    # we should always keep track of the asset term (this is our primary splitter)
    if (asset_idx != 0) and (liable_idx == 0):
        stop_idx1 = asset_idx

    # check the very last, in event our liability term created an early cut-off (e.g. 42352-2003-01-28)
    if (val1 is None) & (val2 is None):
        stop_idx2 = array.size
    
    # partition the array by the enumerated index for asset and liability portions
    lhs = array[:stop_idx1]
    rhs = array[stop_idx1:stop_idx2]
    
    # if either asset or liability side missing, we return None
    if lhs.size == 0 or rhs.size == 0:
        return None
    else:
        return (lhs, rhs, stop_idx1, stop_idx2)

In [3]:
def lineItems(vector:np.ndarray, df:pd.DataFrame):
    """
    Retrieving balance sheet information line item names for s3 files
    ------------------------------------------------------------------------------------
    :param vector: (type numpy.ndarray)
        An array of file names for .csv files from s3 to iterate through
        
    :return: (type tuple)
        Return a tuple of arrays; left is the asset items, and right is the liability & equity items 
    """
    
    # retrieve the asset and liability & equity terms from the dataframe
    response = bsSplit(vector)
    print(vector)
    
    # if response is present we continue 
    if response is not None:
        lhs, rhs, index1, index2 = response      # decompose response object to retrieve index
        
        print('\tAsset Lineitems')
        print(lhs)
        print('\tLiability & Equity Lineitems')
        print(rhs)
        
        # save contents of asset and liability splits to AWS S3 bucket
        dfA = df.iloc[:index1]                  # asset dataframe
        dfL = df.iloc[index1:index2]            # liability and equity dataframe
        
        return (dfA, dfL)
    else:
        return None
   

## Final Main Execution

In [4]:
if __name__ == "__main__":
    
    # initiate s3 bucket and corresponding data folder
    bucket = "ran-s3-systemic-risk"
    
    pdf_data_folder ="Output/X-17A-5-CLEAN-PDFS/"
    png_data_folder ="Output/X-17A-5-CLEAN-PNGS/"
    
    pdf_output_folder = 'Output/X-17A-5-SPLIT-PDFS/'
    png_output_folder = 'Output/X-17A-5-SPLIT-PNGS/'
        
    # Amazon Textract client and Sagemaker session
    s3 = boto3.client('s3')
    session = Session()
    
    # discover all of the data that you want to parse (both PDFS and PNGS)
    pdf_paths = np.array(session.list_s3_files(bucket, pdf_data_folder))[1:]
    pdf_asset_split = np.array(session.list_s3_files(bucket, pdf_output_folder + 'Assets/'))
    pdf_liability_split = np.array(session.list_s3_files(bucket, pdf_output_folder + 'Liability & Equity/'))
    
    png_paths = np.array(session.list_s3_files(bucket, png_data_folder))[1:]
    png_asset_split = np.array(session.list_s3_files(bucket, png_output_folder + 'Assets/'))
    png_liability_split = np.array(session.list_s3_files(bucket, png_output_folder + 'Liability & Equity/'))
    
    # --------------------------------------------------------------------------------------------------
    # PDF PROCESSING (LINE-ITEM SPLIT)
    # --------------------------------------------------------------------------------------------------
    
    # iterate through files from s3 bucket 
    for file in pdf_paths:
        print('\n', file)
        fileName = file.split('/')[-1]                                          # file-name for a given path
        asset_name = pdf_output_folder + 'Assets/' + fileName                   # export path to assets
        liability_name = pdf_output_folder + 'Liability & Equity/' + fileName   # export path to liability and equity
        
        # check to see presence of split files 
        if (asset_name not in pdf_asset_split) or (liability_name not in pdf_liability_split):
        
            # download temporary file from s3 bucket
            s3.download_file(bucket, file, 'temp.csv')
            df = pd.read_csv('temp.csv')

            n = df.columns.size   # the number of columns in read dataframe    

            if n > 1: # if there is more than 1 column we continue examination 

                # all line item for balance sheet (first column)
                arr = df[df.columns[0]].dropna().values     

                # extract line items if possible for both asset and liability terms
                response = lineItems(arr, df)

                # if response not None we decompose each
                if response is not None:
                    # unpack the response object to component parts
                    df_asset, df_liability = response

                    # writing data frame to .csv file (we overwrite file name to save space)
                    df_asset.to_csv(fileName, index=False)
                    with open(fileName, 'rb') as data:
                        s3.put_object(Bucket=bucket, Key=asset_name, Body=data)

                    df_liability.to_csv(fileName, index=False)
                    with open(fileName, 'rb') as data:
                        s3.put_object(Bucket=bucket, Key=liability_name, Body=data)

                    # remove local file after it has been created
                    os.remove(fileName)

            else:
                print('{} incomplete dataframe'.format(file))
                
        else:
            print("We've already downloaded {}".format(fileName))
    
    # --------------------------------------------------------------------------------------------------
    # PNG PROCESSING (LINE-ITEM SPLIT)
    # --------------------------------------------------------------------------------------------------
    
    # iterate through files from s3 bucket 
    for file in png_paths:
        print('\n', file)
        fileName = file.split('/')[-1]                                          # file-name for a given path
        asset_name = png_output_folder + 'Assets/' + fileName                   # export path to assets
        liability_name = png_output_folder + 'Liability & Equity/' + fileName   # export path to liability and equity
        
        # check to see presence of split files 
        if (asset_name not in png_asset_split) or (liability_name not in png_liability_split):
        
            # download temporary file from s3 bucket
            s3.download_file(bucket, file, 'temp.csv')
            df = pd.read_csv('temp.csv')

            n = df.columns.size   # the number of columns in read dataframe    

            if n > 1: # if there is more than 1 column we continue examination 

                # all line item for balance sheet (first column)
                arr = df[df.columns[0]].dropna().values     

                # extract line items if possible for both asset and liability terms
                response = lineItems(arr, df)

                # if response not None we decompose each
                if response is not None:
                    # unpack the response object to component parts
                    df_asset, df_liability = response

                    # writing data frame to .csv file (we overwrite file name to save space)
                    df_asset.to_csv(fileName, index=False)
                    with open(fileName, 'rb') as data:
                        s3.put_object(Bucket=bucket, Key=asset_name, Body=data)

                    df_liability.to_csv(fileName, index=False)
                    with open(fileName, 'rb') as data:
                        s3.put_object(Bucket=bucket, Key=liability_name, Body=data)

                    # remove local file after it has been created
                    os.remove(fileName)

            else:
                print('{} incomplete dataframe'.format(file))
                
        else:
            print("We've already downloaded {}".format(fileName))
    
    # remove local file for storing cleaned data  
    os.remove('temp.csv')
    print('\nAll X-17A-5 files have been downloaded and split between Asset and Liability & Equity terms')


 Output/X-17A-5-CLEAN-PDFS/1101180-2002-02-28.csv
['Cash and cash equivalents'
 'Cash and securities segregated under federal and other regulations'
 'Securities purchased under agreements to resell' 'Securities borrowed'
 'Deposits with clearing organizations (cash of $2,565,447 and securities with a market value of $84,553)'
 'Receivable from brokers, dealers and clearing organizations'
 'Receivable from customers' 'Interest and dividends receivable'
 'Secured demand note with parent' 'Other assets' 'Total Assets'
 'Short-term borrowings' 'Securities sold under agreements to repurchase'
 'Securities loaned' 'Payable to customers'
 'Payable to brokers, dealers and clearing organizations'
 'Interest and dividends payable' 'Other liabilities' 'Total liabilities'
 'Subordinated borrowing with Parent' "Members' capital"
 'Retained earnings' "Total members' capital"
 "Total liabilities and members' capital"]
	Asset Lineitems
['Cash and cash equivalents'
 'Cash and securities segregated un

['Cash and cash equivalents'
 'Cash and securities segregated under Federal and other regulations'
 'Securities purchased under agreements to resell' 'Securities borrowed'
 'Deposits with clearing organizations' 'Receivables from customers'
 'Receivables from broker-dealers and clearing organizations'
 'Securities owned, at fair value' 'Securities pledged as collateral'
 'Interest and dividends receivable' 'Other assets'
 'Short-term borrowings' 'Securities sold under agreements to repurchase'
 'Securities loaned' 'Payables to customers'
 'Payables to broker-dealers and clearing organizations'
 'Obligations to return collateral' 'Interest and dividends payable'
 'Other liabilities' 'Subordinated borrowings' "Member's capital"
 'Retained earnings' "Total liabilities and member's capital"]
	Asset Lineitems
['Cash and cash equivalents'
 'Cash and securities segregated under Federal and other regulations'
 'Securities purchased under agreements to resell' 'Securities borrowed'
 'Deposits w

['Securities sold, not yet purchased, at fair value'
 'Securities sold under agreements to repurchase'
 'Payable to brokers, dealers, and clearing organizations'
 'Loans payable to affiliate' 'Payable to affiliates'
 'Payable for liquidity and order flow' 'Securities loaned'
 'Exchange, clearance, and regulatory fees payable' 'Other liabilities'
 'Total liabilities' "Member's capital"
 "Total liabilities and member's capital"]

 Output/X-17A-5-CLEAN-PDFS/1146184-2021-02-25.csv
We've already downloaded 1146184-2021-02-25.csv

 Output/X-17A-5-CLEAN-PDFS/1215680-2004-03-01.csv
We've already downloaded 1215680-2004-03-01.csv

 Output/X-17A-5-CLEAN-PDFS/1215680-2005-03-01.csv
We've already downloaded 1215680-2005-03-01.csv

 Output/X-17A-5-CLEAN-PDFS/1215680-2006-03-01.csv
We've already downloaded 1215680-2006-03-01.csv

 Output/X-17A-5-CLEAN-PDFS/1215680-2007-03-01.csv
We've already downloaded 1215680-2007-03-01.csv

 Output/X-17A-5-CLEAN-PDFS/1215680-2008-02-29.csv
We've already downloade

['Assets Cash and cash equivalents'
 'Cash and securities segregated and on deposit for federal and other regulations'
 'Trading assets, at fair value'
 'Trading assets, pledged to creditors, at fair value'
 'Total trading assets' 'Investment in affiliate'
 'Securities purchased under agreements to resell' 'Securities borrowed'
 'Clients (net of allowance for doubtful accounts of $17,216)'
 'Brokers and dealers' 'Dividends and interest' 'Fees and other'
 'Receivable from affiliated companies'
 'Office equipment and leasehold improvements, net of accumulated depreciation and amortization of $263,268'
 'Other assets'
 "Liabilities and stockholder's equity Short-term borrowings"
 'Trading liabilities, at fair value'
 'Securities sold under agreements to repurchase' 'Securities loaned'
 'Clients (including free credit balances of $2,811,141)'
 'Brokers and dealers' 'Dividends and interest'
 'Other liabilities and accrued expenses'
 'Accrued compensation and benefits' 'Payable to affiliated


 Output/X-17A-5-CLEAN-PDFS/200565-2007-03-01.csv
['Cash and cash equivalents'
 'Cash and securities segregated and on deposit for federal and other regulations'
 'Trading assets, at fair value'
 'Trading assets, pledged to creditors, at fair value'
 'Total trading assets' 'Securities received as collateral'
 'Securities purchased under agreements to resell' 'Securities borrowed'
 'Clients (net of allowance for doubtful accounts of $8,597)'
 'Brokers and dealers' 'Dividends and interest' 'Fees and other'
 'Receivable from affiliated companies' 'Investment in affiliate'
 'Office equipment and leasehold improvements (net of accumulated depreciation and amortization of $670,011)'
 'Goodwill' 'Intangible (net of accumulated amortization of $4,000)'
 'Other assets' 'Trading liabilities, at fair value'
 'Obligation to return securities'
 'Securities sold under agreements to repurchase' 'Securities loaned'
 'Clients (including free credit balances of $3,534,906)'
 'Brokers and dealers' 'Divid

...


 Output/X-17A-5-CLEAN-PNGS/860220-2008-02-28.csv
['Cash' 'Cash and securities segregated under federal regulations'
 'Securities purchased under agreements to resell'
 'Securities borrowed and securities received as collateral'
 'Securities owned, at fair value (includes $18,654,747 pledged as collateral)'
 'Receivable from brokers, dealers and others' 'Receivable from customers'
 'Accrued interest receivable' 'Investment banking fees receivable'
 'Goodwill' 'Other assets' 'Total assets' 'Short-term borrowings'
 'Securities sold under agreements to repurchase'
 'Securities loaned and obligation to return securities received as collateral'
 'Securities sold, not yet purchased, at fair value'
 'Payable to brokers, dealers and others' 'Payable to customers'
 'Accrued interest payable'
 'Accrued expenses, compensation and other liabilities'
 'Commitments and contingencies (Notes 10 and 11)'
 'Liabilities subordinated to claims of general creditors'
 'Common units, 10,000 authorized, issue

['Cash and cash equivalents'
 'Cash and securities deposited with clearing organizations or segregated in compliance with federal regulations'
 'Securities purchased under agreements to resell' 'Securities borrowed'
 'Securities received as collateral' 'Customers'
 'Brokers, dealers and others' 'Interest and dividends' 'Other assets'
 'TOTAL ASSETS' 'Short-term borrowings'
 'Securities sold under agreements to repurchase' 'Securities loaned'
 'Obligation to return securities received as collateral' 'Customers'
 'Brokers, dealers and others' 'Interest and dividends'
 'Other liabilities and accrued expenses' 'Subordinated liabilities'
 'Preferred stock, $1.00 par value; 9,000 shares authorized (Series A Preferred Stock, $1,500 liquidation preference); 850 shares outstanding'
 'and outstanding' 'Paid-in capital' 'Retained earnings'
 "Total Stockholders' Equity" "TOTAL LIABILITIES AND STOCKHOLDERS' EQUITY"]
	Asset Lineitems
['Cash and cash equivalents'
 'Cash and securities deposited with 


 Output/X-17A-5-CLEAN-PNGS/874362-2008-01-31.csv
['Cash and cash equivalents'
 'Cash and securities deposited with clearing organizations or segregated in compliance with federal regulations'
 'Securities purchased under agreements to resell' 'Securities borrowed'
 'Securities received as collateral' 'Customers'
 'Brokers, dealers and others' 'Interest and dividends' 'Other assets'
 'TOTAL ASSETS' 'Short-term borrowings from third parties'
 'Short-term borrowings from Ultimate Parent'
 'Securities sold under agreements to repurchase' 'Securities loaned'
 'Obligation to return securities received as collateral' 'Customers'
 'Brokers, dealers and others' 'Interest and dividends'
 'Other liabilities and accrued expenses' 'Subordinated liabilities'
 '330 shares outstanding' 'and outstanding' 'Paid-in capital'
 'Retained earnings' "Total Stockholders' Equity"
 'TOTAL LIABILITIES AND STOCKHOLDERS EQUITY']
	Asset Lineitems
['Cash and cash equivalents'
 'Cash and securities deposited with cle


 Output/X-17A-5-CLEAN-PNGS/874362-2014-03-05.csv
['Cash'
 'Cash and securities segregated under federal and other regulations'
 'Securities borrowed' 'Customers'
 'Brokers, dealers, clearing organizations and others'
 'Interest and dividends' 'Other assets' 'Total assets' 'Borrowings'
 'Securities sold under repurchase agreements' 'Securities loaned'
 'Customers' 'Brokers, dealers and others' 'Interest and dividends'
 'Other liabilities and accrued expenses' 'Subordinated liabilities'
 'Total liabilities' "Total stockholders' equity"
 "Total liabilities and stockholders' equity"]
	Asset Lineitems
['Cash'
 'Cash and securities segregated under federal and other regulations'
 'Securities borrowed' 'Customers'
 'Brokers, dealers, clearing organizations and others'
 'Interest and dividends' 'Other assets' 'Total assets']
	Liability & Equity Lineitems
['Borrowings' 'Securities sold under repurchase agreements'
 'Securities loaned' 'Customers' 'Brokers, dealers and others'
 'Interest and di

['Cash and cash equivalents'
 'Cash and securities segregated and on deposit for regulatory and other purposes'
 'Pledged as collateral' 'Not pledged as collateral'
 'Securities purchased under agreements to resell' 'Securities borrowed'
 'Brokers, dealers and clearing organizations' 'Customers'
 'Receivables from affiliates' 'Others'
 'Property, equipment and leasehold improvements (net of accumulated depreciation and amortization of $129 in 2002)'
 'Other assets'
 'Excess of cost over fair value of net asseis acquired (net of accumulated amortization of $134 in 2002)'
 'Total assets' 'Short-term debt'
 'Securities and other financial instruments sold but not yet purchased'
 'Securities sold under agreements to repurchase' 'Securities loaned'
 'Other secured financing' 'Advances from Holdings and other affiliates'
 'Brokers, dealers and clearing organizations' 'Customers'
 'Accrued liabilities and other payables' 'Senior notes'
 'Subordinated indebtedness' 'Total liabilities']
	Asset 

In [5]:
# # runnning singular analysis on a particular dataset
# # s3.download_file('ran-s3-systemic-risk', 'Output/X-17A-5-CLEAN-PNGS/782124-2014-03-05.csv', 'temp.csv')
# # s3.download_file('ran-s3-systemic-risk', 'Output/X-17A-5-CLEAN-PDFS/42352-2003-01-28.csv', 'temp.csv')
# s3.download_file('ran-s3-systemic-risk', 'Output/X-17A-5-CLEAN-PDFS/782124-2013-03-01.csv', 'temp.csv')

# df = pd.read_csv('temp.csv')
# os.remove('temp.csv')

# # all line item for balance sheet (first column)
# arr = df[df.columns[0]].dropna().values     

# # extract line items if possible for both asset and liability terms
# response = lineItems(arr, df)